In [1]:
# Detecta la board en cada fotograma.
# Va cogiendo los 20 últimos fotogramas de la board y su resultado más común es el state de la board.
# Si se produce un cambio entre los dos últimos states te devuelve el output donde el robot tiene que poner el token (de forma random).
# Si los 3 tokens están en partida, se quita un token de una casilla y se pone en otra.
#

# DIFERENCIAS
# Código intentando crear el tablero desde un solo starting-point, habiendo definido la distancia entre cada celda primero

# ERRORES:
# Works with out check win function

import cv2
import numpy as np
from collections import defaultdict
import random
import time
from PIL import Image
import serial.tools.list_ports
import pydobot

# Get the port name of the Dobot arm
available_ports = list(serial.tools.list_ports.comports())
port = available_ports[1].device

# Create a Dobot instance and connect to the arm
dobot = pydobot.Dobot(port=port)

# Calibration coordinates for the board

start_coordinates = [242.37692260742188, 11.542285919189453, -65.57083892822266, 0]

calibration_celda_1 = [167.32986450195312, -272.492431640625, -59.79207229614258, 0]
calibration_celda_2 = [124.93033599853516, -227.24774169921875, -62.601585388183594, 0]
calibration_celda_3 = [77.44032287597656, -181.2471923828125, -63.24043273925781, 0]
calibration_celda_4 = [216.61453247070312, -226.2322998046875, -59.78167724609375, 0]
calibration_celda_5 = [173.50437927246094, -181.20803833007812, -61.63672637939453, 0]
calibration_celda_6 = [129.7561798095703, -138.81768798828125, -62.505889892578125, 0]
calibration_celda_7 = [265.1841125488281, -179.68240356445312, -59.60185623168945, 0]
calibration_celda_8 = [220.31028747558594, -136.8333282470703, -61.52882385253906, 0]
calibration_celda_9 = [175.16598510742188, -91.30010986328125, -62.3995361328125, 0]

calibration_token_1 = [235.37603759765625, -20.93382453918457, -63.679481506347656, 0]
calibration_token_2 = [266.7563171386719, -51.0143928527832, -63.26238250732422, 0]
calibration_token_3 = [296.2026672363281, -80.15022277832031, -63.648048400878906, 0]

calibration_home = [210.09783935546875, -3.7860727310180664, 116.83699035644531, 0]
calibration_middle = [165.581787109375, -171.8178253173828, 116.74826049804688, 0]

diffs = []
for celda in [calibration_celda_1, calibration_celda_2, calibration_celda_3, calibration_celda_4, calibration_celda_5, calibration_celda_6, calibration_celda_7, calibration_celda_8, calibration_celda_9, calibration_token_1, calibration_token_2, calibration_token_3, calibration_home, calibration_middle]:
    diff = []
    for i in range(3):
        diff.append(celda[i] - start_coordinates[i])
    diffs.append(diff)

# Here you input the coordinates of the start point
start = [281.4211120605469, 0.0, -41.053348541259766, 0]

# Create new variables for each new cell
celda_1 = [start[0]+diffs[0][0], start[1]+diffs[0][1], start[2]+diffs[0][2], 0]
celda_2 = [start[0]+diffs[1][0], start[1]+diffs[1][1], start[2]+diffs[1][2], 0]
celda_3 = [start[0]+diffs[2][0], start[1]+diffs[2][1], start[2]+diffs[2][2], 0]
celda_4 = [start[0]+diffs[3][0], start[1]+diffs[3][1], start[2]+diffs[3][2], 0]
celda_5 = [start[0]+diffs[4][0], start[1]+diffs[4][1], start[2]+diffs[4][2], 0]
celda_6 = [start[0]+diffs[5][0], start[1]+diffs[5][1], start[2]+diffs[5][2], 0]
celda_7 = [start[0]+diffs[6][0], start[1]+diffs[6][1], start[2]+diffs[6][2], 0]
celda_8 = [start[0]+diffs[7][0], start[1]+diffs[7][1], start[2]+diffs[7][2], 0]
celda_9 = [start[0]+diffs[8][0], start[1]+diffs[8][1], start[2]+diffs[8][2], 0]

token_1 = [start[0]+diffs[9][0], start[1]+diffs[9][1], start[2]+diffs[9][2], 0]
token_2 = [start[0]+diffs[10][0], start[1]+diffs[10][1], start[2]+diffs[10][2], 0]
token_3 = [start[0]+diffs[11][0], start[1]+diffs[11][1], start[2]+diffs[11][2], 0]

home = [start[0]+diffs[12][0], start[1]+diffs[12][1], start[2]+diffs[12][2], 0]
middle = [start[0]+diffs[13][0], start[1]+diffs[13][1], start[2]+diffs[13][2], 0]

dobot.move_to(home[0], home[1], home[2], home[3])


# Funciones para saber en que cuadro está un token

def get_square_number(row, col):
    # row and col are 0-indexed, so add 1 to convert to 1-indexed square number
    return row * 3 + col + 1

def get_square_new_number(row, col):
    # row and col are 0-indexed, so add 1 to convert to 1-indexed square number
    return row * 3 + col + 1

# Function to check for game_over 

def check_win(board):
    # Check rows
    for row in board:
        if len(set(row)) == 1 and row[0] != 0:
            return True

    # Check columns
    for col in range(3):
        if len(set([board[row][col] for row in range(3)])) == 1 and board[0][col] != 0:
            return True

    # Check diagonals
    if len(set([board[i][i] for i in range(3)])) == 1 and board[0][0] != 0:
        return True
    
    if len(set([board[i][2-i] for i in range(3)])) == 1 and board[0][2] != 0:
        return True

    return False


height, width = 450, 350

# Define the four corners of the tic tac toe board in the original frame

coordinates = [[694, 665], [1170, 666], [524, 1048], [1422, 1028]]


src_points = np.float32(coordinates)

# Define the four corners of the destination image
dst_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]])


# Initialize a list that will contain the boards
boards = []

# Initialize a list that will contain the states of the board 
results = [[[0, 0, 0], [0, 0, 0], [0, 0, 0]]]

result = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

# count of how many tokens you have placed on the board
tokens_placed = 0

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    transformed_frame = cv2.warpPerspective(frame, M, (width, height))

    # Code to rotate the image 180 degrees
    pil_image = Image.fromarray(transformed_frame)
    transformed_frame = pil_image.rotate(180)
    transformed_frame = np.array(transformed_frame)

     # Convert the transformed frame to grayscale
    gray = cv2.cvtColor(transformed_frame, cv2.COLOR_BGR2GRAY)

    # Adjust the Canny edge detection thresholds
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(gray, low_threshold, high_threshold, apertureSize=3)

    # Find contours in the image
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize an empty list to store the coordinates of the squares
    squares = []

    # Loop through the contours
    for cnt in contours:
        # Approximate the contour with a polygon
        approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)
        # If the polygon has four vertices and is convex, it could be a rectangle
        if len(approx) == 4 and cv2.isContourConvex(approx):
            # Calculate the bounding box of the polygon
            x, y, w, h = cv2.boundingRect(approx)
            # Draw a green rectangle around the rectangle
            cv2.rectangle(transformed_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
             # Add the coordinates of the rectangle to the list
            squares.append((x, y, x+w, y+h))

    # Sort the squares from left to right and top to bottom
    squares = sorted(squares, key=lambda c: (c[1] // 124) * 3 + (c[0] // 107))


    # Initialize a 3x3 grid of zeros to represent the tic tac toe board
    board = np.zeros((3, 3), dtype=np.int) 

    # Check if there are exactly 9 squares
    if len(squares) != 9:
        #print("Error: Detected {} squares instead of 9".format(len(squares)))
        exit()
    
    else:
    # Loop through the squares and fill in the corresponding cell of the grid
        for i, square in enumerate(squares):
            row = i // 3
            col = i % 3
            x1, y1, x2, y2 = square
            # Add the square to the grid
            board[row, col] = 0

            # Crop the image to the bounding box of the square
            square_img = transformed_frame[y1:y2, x1:x2]

            # Convert the cropped image to grayscale
            square_gray = cv2.cvtColor(square_img, cv2.COLOR_BGR2GRAY)

            # Aplicar un filtro gaussiano para reducir el ruido
            blur = cv2.GaussianBlur(gray, (5, 5), 0)

            # Apply the Hough circle transform to detect circles
            circles = cv2.HoughCircles(square_gray, cv2.HOUGH_GRADIENT, dp=1, minDist=20, param1=50, param2=12, minRadius=20, maxRadius=30)


            if circles is not None:

                circles = circles[0]
                for (x, y, r) in circles:
                    # Extraer la región del círculo
                    circle_region = square_img[int(y-r):int(y+r), int(x-r):int(x+r)]
                    if not circle_region.any():
                        continue
                    circle_hsv = cv2.cvtColor(circle_region, cv2.COLOR_BGR2HSV)
                    
                    
                    # Definir rangos de color para rojo y verde
                    lower_red = (169, 148, 180)
                    upper_red = (176, 193, 235)
                    lower_green = (55, 99, 158)
                    upper_green = (64, 142, 236)
                    
                    # Segmentar el color del círculo utilizando los rangos de color definidos
                    mask_red = cv2.inRange(circle_hsv, lower_red, upper_red)
                    mask_green = cv2.inRange(circle_hsv, lower_green, upper_green)
                    
                    # Contar los píxeles de cada máscara para determinar el color predominante
                    count_red = cv2.countNonZero(mask_red)
                    count_green = cv2.countNonZero(mask_green)
                    
                    # Imprimir el resultado
                    if count_red > count_green:
                        board[row, col] = 1 
                                            
                    else:
                        board[row, col] = 2 
            
        # Print the grid
        # print(board)

        # Change the type of the board from a string to a list
        board_list = board.tolist()
        # print(board_list)

        # Append the all the new boards to a list
        boards.append(board_list)
        #print(len(boards))
        freq = defaultdict(int)

        if len(boards) == 20:
            for board in boards:
                freq[str(board)] += 1

            # find board with highest frequency
            max_board = max(freq, key=freq.get)
            result = eval(max_board)  # convert string representation back to list

            # print(result)
            results.append(result)

            # Check if its a win after user moves
            if check_win(result) == True:
                wait_time = 30
                start_time = time.time()
                while (time.time() - start_time) < wait_time:
                    print ("GAME OVER")

            if len(results) >= 2:
                if results[-1] != results[-2]:

                    # place token on the board at random location
                    if tokens_placed < 3:
                        # choose a random row and column
                        while True:
                            row = random.randint(0, 2)
                            col = random.randint(0, 2)

                            # check if the cell is empty
                            if result[row][col] == 0:
                                break

                        # place your token in the cell
                        result[row][col] = 2
                        

                        position = get_square_number(row, col)
                        
                        if tokens_placed == 0:
                            dobot.move_to(token_1[0], token_1[1], token_1[2], token_1[3])

                        if tokens_placed == 1:
                            dobot.move_to(token_2[0], token_2[1], token_2[2], token_2[3])

                        if tokens_placed == 2:
                            dobot.move_to(token_3[0], token_3[1], token_3[2], token_3[3])
                        
                        tokens_placed += 1

                        dobot.suck(True)
                        dobot.move_to(middle[0], middle[1], middle[2], middle[3])

                        if position == 1:
                            dobot.move_to(celda_1[0], celda_1[1], celda_1[2], celda_1[3])

                        if position == 2:
                            dobot.move_to(celda_2[0], celda_2[1], celda_2[2], celda_2[3])

                        if position == 3:
                            dobot.move_to(celda_3[0], celda_3[1], celda_3[2], celda_3[3])

                        if position == 4:
                            dobot.move_to(celda_4[0], celda_4[1], celda_4[2], celda_4[3])

                        if position == 5:
                            dobot.move_to(celda_5[0], celda_5[1], celda_5[2], celda_5[3])

                        if position == 6:
                            dobot.move_to(celda_6[0], celda_6[1], celda_6[2], celda_6[3])

                        if position == 7:
                            dobot.move_to(celda_7[0], celda_7[1], celda_7[2], celda_7[3])

                        if position == 8:
                            dobot.move_to(celda_8[0], celda_8[1], celda_1[2], celda_8[3])
                        
                        if position == 9:
                            dobot.move_to(celda_9[0], celda_9[1], celda_9[2], celda_9[3])
                        
                        dobot.suck(False)

                        dobot.move_to(middle[0], middle[1], middle[2], middle[3])
                        dobot.move_to(home[0], home[1], home[2], home[3])


                    # if all tokens have been placed, move one to a different location
                    else:
                        # choose a random cell with your token
                        while True:
                            row = random.randint(0, 2)
                            col = random.randint(0, 2)
                            if result[row][col] == 2:
                                break
                        
                        # choose a random empty cell
                        while True:
                            new_row = random.randint(0, 2)
                            new_col = random.randint(0, 2)
                            if result[new_row][new_col] == 0:
                                break
                        
                        # move the token to the new cell
                        result[new_row][new_col] = 2
                        result[row][col] = 0

                        # get the cells with the number
                        position_1 = get_square_number(row, col)
                        position_2 = get_square_new_number(new_row, new_col)

                        dobot.move_to(middle[0], middle[1], middle[2], middle[3])

                        if position_1 == 1:
                            dobot.move_to(celda_1[0], celda_1[1], celda_1[2], celda_1[3])

                        if position_1 == 2:
                            dobot.move_to(celda_2[0], celda_2[1], celda_2[2], celda_2[3])

                        if position_1 == 3:
                            dobot.move_to(celda_3[0], celda_3[1], celda_3[2], celda_3[3])

                        if position_1 == 4:
                            dobot.move_to(celda_4[0], celda_4[1], celda_4[2], celda_4[3])

                        if position_1 == 5:
                            dobot.move_to(celda_5[0], celda_5[1], celda_5[2], celda_5[3])

                        if position_1 == 6:
                            dobot.move_to(celda_6[0], celda_6[1], celda_6[2], celda_6[3])

                        if position_1 == 7:
                            dobot.move_to(celda_7[0], celda_7[1], celda_7[2], celda_7[3])

                        if position_1 == 8:
                            dobot.move_to(celda_8[0], celda_8[1], celda_1[2], celda_8[3])
                        
                        if position_1 == 9:
                            dobot.move_to(celda_9[0], celda_9[1], celda_9[2], celda_9[3])

                        dobot.suck(True)
                        dobot.move_to(middle[0], middle[1], middle[2], middle[3])

                        if position_2 == 1:
                            dobot.move_to(celda_1[0], celda_1[1], celda_1[2], celda_1[3])

                        if position_2 == 2:
                            dobot.move_to(celda_2[0], celda_2[1], celda_2[2], celda_2[3])

                        if position_2 == 3:
                            dobot.move_to(celda_3[0], celda_3[1], celda_3[2], celda_3[3])

                        if position_2 == 4:
                            dobot.move_to(celda_4[0], celda_4[1], celda_4[2], celda_4[3])

                        if position_2 == 5:
                            dobot.move_to(celda_5[0], celda_5[1], celda_5[2], celda_5[3])

                        if position_2 == 6:
                            dobot.move_to(celda_6[0], celda_6[1], celda_6[2], celda_6[3])

                        if position_2 == 7:
                            dobot.move_to(celda_7[0], celda_7[1], celda_7[2], celda_7[3])

                        if position_2 == 8:
                            dobot.move_to(celda_8[0], celda_8[1], celda_1[2], celda_8[3])
                        
                        if position_2 == 9:
                            dobot.move_to(celda_9[0], celda_9[1], celda_9[2], celda_9[3])
                        
                        dobot.suck(False)

                        dobot.move_to(middle[0], middle[1], middle[2], middle[3])
                        dobot.move_to(home[0], home[1], home[2], home[3])

                    
                    print("move here: ", result)
                    results.append(result)

                    
                    # Giving time to the dobot to pick the token and move it to the cell
                    wait_time = 15

                    start_time = time.time()
                    print("ROBOT MOVES...")
                    
                    while (time.time() - start_time) < wait_time:
                        print(result, "...")
                    print("DONE!")
                    
                    # Check if its a win after robot moves
                    if check_win(result) == True:
                        wait_time = 30
                        start_time = time.time()
                        while (time.time() - start_time) < wait_time:
                            print ("GAME OVER")
        
            # print the updated board
            print(result)

            boards = []

    cv2.imshow('frame', transformed_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.waitKey(0)
cv2.destroyAllWindows()

<ipython-input-1-c426e2bbdd38>:178: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  board = np.zeros((3, 3), dtype=np.int)


move here:  [[0, 0, 0], [0, 0, 0], [0, 2, 1]]
ROBOT MOVES...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 2, 1]] ...
[[0, 0, 0], [0, 0, 0], [0, 

SerialException: write failed: [Errno 6] Device not configured

: 